# 7506 - Organizacion de Datos - TP N°2

# 1. Librerias

In [78]:
#librerías, no es necesario volverlas a importar

import os
import pandas as pd

import nltk, re, string, collections
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

%run ../0_Data/Util.ipynb

[nltk_data] Downloading package punkt to /home/celeste/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/celeste/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/celeste/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# 2. Datos

In [14]:
#Dataset de entranamiento
df_train = pd.read_csv(os.path.join('..','0_Data','dataset','train.csv'),index_col='id', encoding='utf8')

#Dataset de test
df_test = pd.read_csv(os.path.join('..','0_Data','dataset','test.csv'),index_col='id', encoding='utf8')

In [15]:
df_train.head()

,keyword,location,text,target
id,,,,
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [16]:
df_test.head()


,keyword,location,text
id,,,
0,NaN,NaN,Just happened a terrible car crash
2,NaN,NaN,"Heard about #earthquake is different cities, s..."
3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


# 3. Preprocesamiento

In [17]:
def eliminar_puntuacion_excepto_puntos(text):
    punctuationNoPeriod = re.sub("\.","",string.punctuation)
    return text.translate(str.maketrans('','',punctuationNoPeriod))

In [73]:
def preprocesar_df(df, col_name, clean_col_name):
    df[clean_col_name] = df[col_name].apply(lambda x: minuscula (x))\
                                    .apply(lambda x: remover_url(x))\
                                    .apply(lambda x: remover_usuario(x))\
                                    .apply(lambda x: remover_emoji(x))\
                                    .apply(lambda x: expandir_abreviatura(x))\
                                    .apply(lambda x: expandir_contraccion(x))\
                                    .apply(lambda x: remover_caracter_especial(x))\
                                    .apply(lambda x: remover_tag_html(x))\
                                    .apply(lambda x: remover_acento(x))\
                                    .apply(lambda x: remover_punto(x))\
                                    .apply(lambda x: remover_numero(x))\
                                    .apply(lambda x: remover_espacio_extra(x))\
                                    .apply(lambda x: remover_stop_word(x))\
                                    .apply(lambda x: lematizar(x))


In [79]:
%time preprocesar_df (df_train,'text', 'texto_preprocesado')

CPU times: user 10.8 s, sys: 675 ms, total: 11.5 s
Wall time: 11.6 s


In [20]:
df_train.head()

,keyword,location,text,target,texto_preprocesado
id,,,,,
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquake may allah forgive u
4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask . canada
5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,resident asked shelter place notified officer ...
6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfire evacuation order calif...
7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,got sent photo ruby alaska smoke wildfire pour...


In [21]:
%time preprocesar_df (df_test,'text', 'texto_preprocesado')

CPU times: user 4.68 s, sys: 248 ms, total: 4.93 s
Wall time: 4.93 s


In [22]:
df_test.head()

,keyword,location,text,texto_preprocesado
id,,,,
0,NaN,NaN,Just happened a terrible car crash,happened terrible car crash
2,NaN,NaN,"Heard about #earthquake is different cities, s...",heard earthquake different city stay safe ever...
3,NaN,NaN,"there is a forest fire at spot pond, geese are...",forest fire spot pond goose fleeing across str...
9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,apocalypse lighting . spokane wildfire
11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,typhoon soudelor kill china taiwan


In [23]:
df_train_preprocesado = df_train[['texto_preprocesado','target']]
df_train_preprocesado.head()

,texto_preprocesado,target
id,,
1,deed reason earthquake may allah forgive u,1
4,forest fire near la ronge sask . canada,1
5,resident asked shelter place notified officer ...,1
6,people receive wildfire evacuation order calif...,1
7,got sent photo ruby alaska smoke wildfire pour...,1


In [81]:
df_test_preprocesado = df_test['texto_preprocesado']

In [80]:
x_train_original = df_train['text']

x_train_preprocesado = df_train['texto_preprocesado']

y_train = df_train['target']

In [26]:
x_test_original = df_test['text']

x_test_preprocesado = df_test['texto_preprocesado']

# 4. Vectorizacion

## 4.1  Obtencion de NGramas, análisis de distintas herramientas

In [27]:
def get_ngrams(documents, size):
    word_vectorizer = CountVectorizer()
    sparse_matrix = word_vectorizer.fit_transform(documents)
    frequencies = sum(sparse_matrix).data
    #frequencies = sum(sparse_matrix).toarray()[0]
    df = pd.DataFrame(frequencies, index=word_vectorizer.get_feature_names(), columns=['frequency'])
    df = df.sort_values(by='frequency', ascending=False)
    return df

In [28]:
%time get_ngrams(df_train['texto_preprocesado'],1)

CPU times: user 1.87 s, sys: 0 ns, total: 1.87 s
Wall time: 1.87 s


,frequency
lglorg,353
lily,348
landi,300
literally,256
leelanau,234
...,...
fitz,1
fittscott,1
fist,1
fishing,1


In [70]:
def documentNgrams(documents, size, count=None):
    ngrams_all = []
    for document in documents:
        tokens = document.split()
        if len(tokens) < size:
            continue
        else:
            output = list(ngrams(tokens, size))
        for ngram in output:
            ngrams_all.append(" ".join(ngram))
    cnt_ngram = Counter()
    for word in ngrams_all:
        cnt_ngram[word] += 1
    df = pd.DataFrame.from_dict(cnt_ngram, orient='index').reset_index()
    df = df.rename(columns={'index':'words', 0:'count'})
    df = df.sort_values(by='count', ascending=False)
    if (count == None):
        return df
    else:
        return(df.head(count))
    


In [68]:
#elijo este que da mejor tiempo de ejecucion y ademas el otro saltea un par de casos. 
# mas adelante se analiza que los casos que saltea son los de letras solas, por ejemplo "u are" 

%time documentNgrams(df_train['texto_preprocesado'],1, 100)

CPU times: user 84.1 ms, sys: 0 ns, total: 84.1 ms
Wall time: 84.5 ms


,words,count
13,.,3106
61,...,1066
8,fire,352
124,like,347
299,amp,300
...,...,...
491,even,72
410,another,71
575,really,71
2242,hostage,70


In [31]:
df_train['tokens'] = df_train['texto_preprocesado'].apply(lambda x : x.split())
df_train.head()

,keyword,location,text,target,texto_preprocesado,tokens
id,,,,,,
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquake may allah forgive u,"[deed, reason, earthquake, may, allah, forgive..."
4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask . canada,"[forest, fire, near, la, ronge, sask, ., canada]"
5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,resident asked shelter place notified officer ...,"[resident, asked, shelter, place, notified, of..."
6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfire evacuation order calif...,"[people, receive, wildfire, evacuation, order,..."
7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,got sent photo ruby alaska smoke wildfire pour...,"[got, sent, photo, ruby, alaska, smoke, wildfi..."


In [32]:
tokens = df_train.explode('tokens')
tokens.head()


,keyword,location,text,target,texto_preprocesado,tokens
id,,,,,,
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquake may allah forgive u,deed
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquake may allah forgive u,reason
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquake may allah forgive u,earthquake
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquake may allah forgive u,may
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquake may allah forgive u,allah


In [33]:
token_list = tokens['tokens'].value_counts().reset_index()

In [34]:
token_list['len'] = token_list['index'].apply(lambda x: len(x))
token_list.head().sort_values('len')

,index,tokens,len
0,.,3106,1
1,...,1066,3
4,amp,300,3
2,fire,352,4
3,like,347,4


In [35]:
token_list



,index,tokens,len
0,.,3106,1
1,...,1066,3
2,fire,352,4
3,like,347,4
4,amp,300,3
...,...,...,...
13535,aaaaaand,1,8
13536,quiet,1,5
13537,petty,1,5
13538,mascara,1,7


In [36]:
vect_list = get_ngrams(df_train['texto_preprocesado'],1).reset_index()['index'].to_list()

In [37]:
token_list.loc[token_list['index'].apply(lambda x : x not in vect_list),:]

,index,tokens,len
0,.,3106,1
1,...,1066,3
17,u,151,1
340,w,37,1
363,x,36,1
...,...,...,...
13339,guns..,1,6
13416,drought.check,1,13
13512,updatem.e,1,9
13516,killers..,1,9


## 4.2  Obtencion de los 100 BiGramas mas frecuentes para todos los tweets

In [41]:
%time bigrams_all_tweets = documentNgrams(df_train['texto_preprocesado'],2, 100)

CPU times: user 89.5 ms, sys: 0 ns, total: 89.5 ms
Wall time: 90.7 ms


In [64]:
bigrams_all_tweets.head(10)

,words,count
38553,suicide bomber,60
7223,burning building,58
114,... .,56
1630,laughing loud,53
5762,body bag,50
1014,look like,49
88,gon na,43
453,prime minister,42
2278,northern california,41
1778,liked video,41


In [43]:
bigrams_list = bigrams_all_tweets.words.to_list()

In [47]:
word_vectorizer_100 = CountVectorizer(ngram_range=(2, 2), vocabulary = bigrams_list)
sparse_matrix = word_vectorizer_100.fit_transform(x_train_preprocesado.head(10))

In [48]:
print (sparse_matrix.toarray()[1][31])

1


In [49]:
bigrams_list[31]

'forest fire'

In [50]:
x_train_preprocesado.iloc[1]

'forest fire near la ronge sask . canada'

## 4.3  Obtencion de los 1000 BiGramas mas frecuentes para todos los tweets

In [53]:
%time bigrams_all_tweets_1000 = documentNgrams(df_train['texto_preprocesado'],2, 1000)

CPU times: user 98.3 ms, sys: 0 ns, total: 98.3 ms
Wall time: 98.3 ms


In [54]:
bigrams_list_1000 = bigrams_all_tweets_1000.words.to_list()

In [55]:
word_vectorizer_1000 = CountVectorizer(ngram_range=(2, 2), vocabulary = bigrams_list_1000)

In [66]:
vectorizer = CountVectorizer(ngram_range = (2,2), max_features = 5) 
vectorizer.fit_transform(x_train_preprocesado)

<7613x5 sparse matrix of type '<class 'numpy.int64'>'
	with 268 stored elements in Compressed Sparse Row format>

In [67]:
vectorizer.get_feature_names()

['body bag',
 'burning building',
 'laughing loud',
 'look like',
 'suicide bomber']

## 4.4  Obtencion de los BiGramas mas frecuentes para los tweets que son desastres 

In [91]:
%time bigrams_true_tweets = documentNgrams(df_train.loc[(df_train['target']== 1),'texto_preprocesado'],2)

CPU times: user 58.8 ms, sys: 7.74 ms, total: 66.6 ms
Wall time: 72.6 ms


In [92]:
bigrams_true_tweets

,words,count
15593,suicide bomber,60
851,northern california,41
13522,oil spill,38
1536,california wildfire,36
227,prime minister,36
...,...,...
6861,nevada exceptional,1
6862,exceptional drought,1
6863,drought steady,1
6864,steady nv,1


## 4.5  Obtencion de los TriGramas mas frecuentes para los tweets que son desastres 

In [94]:
%time trigrams_true_tweets = documentNgrams(df_train.loc[(df_train['target']== 1),'texto_preprocesado'],3)

CPU times: user 45.4 ms, sys: 65 µs, total: 45.5 ms
Wall time: 45.4 ms


In [95]:
trigrams_true_tweets

,words,count
14862,suicide bomber detonated,30
10042,northern california wildfire,29
13494,latest home razed,28
13495,home razed northern,28
14863,bomber detonated bomb,28
...,...,...
6410,true story real,1
6411,story real pain,1
6412,real pain amp,1
6413,pain amp suffering,1


## 4.6  Obtencion del vocabulario de los tweets que son desastres naturales  

In [97]:
%time vocabulary_true_tweets = documentNgrams(df_train.loc[(df_train['target']== 1),'texto_preprocesado'],1)

CPU times: user 67.8 ms, sys: 0 ns, total: 67.8 ms
Wall time: 67.6 ms


In [98]:
vocabulary_true_tweets

,words,count
8,fire,265
321,news,139
348,via,121
45,disaster,118
26,california,115
...,...,...
3753,hubert,1
3752,dakota,1
3751,economics,1
3750,learned,1


In [99]:
vocabulary_true_tweets_original = documentNgrams(df_train.loc[(df_train['target']== 1),'text'],1)

In [100]:
vocabulary_true_tweets_original

,words,count
3,the,1051
25,in,1037
5,of,888
55,a,730
23,to,706
...,...,...
7028,https://t.co/X17fUBQBGG,1
7029,@_jeesss_,1
7030,@Ethereal_7,1
7033,medic,1
